<!--<br/><br/><br/><br/><br/><br/><br/><br/>
<font size=10>https://tinyurl.com/bg2mlBoF</font><br/>
<font size=1>https://www.dropbox.com/sh/0bmxic46xg5k6my/AAAVdt_AvCJZPfzjb8jmNvxda?dl=0</font>
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>
-->

![Beginners Guide to Machine Learning in Astronomy](Intro.png "Beginners Guide to Machine Learning in Astronomy")

![https://creativecommons.org/licenses/by-nc-sa/4.0/](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "https://creativecommons.org/licenses/by-nc-sa/4.0/")
<font size=1>
presented by Kai Polsterer, HITS gGmbH at Ecole Doctorale 182 in Strassbourg, France. Not taking responsibility for anything including proper functionality and content of links.
</font>

<br/><br/><br/><br/><font color="113388" size="6">1. Measuring redshift based on photometric measurements</font>
<br/>
<font size=4>
Before starting with the actual machine learning part, we need an example application.
    
</font>

<center><h1>How about estimating the distance of quasars?</h1></center>

<br/><br/>
<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">
    
read more about quasars

* https://en.wikipedia.org/wiki/Quasar
* https://www.britannica.com/science/quasar
* http://adsabs.harvard.edu/abs/1993ARA%26A..31..473A

read more about photometric redshifts
    
* https://arxiv.org/abs/1805.12574
</div>

<br/><br/>

![Photometric Redshifts](PhotoZ.png "Photometric Redshifts")

<br/><br/><br/><br/><font color="113388" size="6">2. Machine Learning or<br/><br/><center>"the quest of finding a good predictive model"</center></font><br/><br/>

<font size=4>
    
given ...


* the variety of input data
* the options to transform or preprocess the data
* assumptions on noise and measurement errors
* available algorithms and their hyperparameters
* different similarity measures
* different scoring and evaluation methods

the question is ...

* what is a predictive model?
* what is a good model?
* is the model proper generalizing?

</font>

<br/><br/><br/><br/><font color="113388" size="6">3. Classification with explicit criterion</font><br/><br/>

<font size=4>
Before we can start estimating distances, we have to learn how to classify ...
</font>

<br/><br/>
<center><h1>star vs. galaxy</h1></center><br/><br/><br/>

<font size=4>
Let's use data from Sloan Digital Sky Servey(SDSS) to do some experiments.<br/><br/>
</font>


<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">
    
read more about SDSS data source
    
* https://www.sdss.org/dr14/algorithms/classify/
* https://skyserver.sdss.org/dr14/en/help/browser/browser.aspx#&&history=description+SpecPhotoAll+U
    
Acknowledgments

<font size=1>
Funding for the Sloan Digital Sky Survey IV has been provided by the Alfred P. Sloan Foundation, the U.S. Department of Energy Office of Science, and the Participating Institutions. SDSS acknowledges support and resources from the Center for High-Performance Computing at the University of Utah. The SDSS web site is https://www.sdss.org/.

SDSS is managed by the Astrophysical Research Consortium for the Participating Institutions of the SDSS Collaboration including the Brazilian Participation Group, the Carnegie Institution for Science, Carnegie Mellon University, the Chilean Participation Group, the French Participation Group, Harvard-Smithsonian Center for Astrophysics, Instituto de Astrofísica de Canarias, The Johns Hopkins University, Kavli Institute for the Physics and Mathematics of the Universe (IPMU) / University of Tokyo, the Korean Participation Group, Lawrence Berkeley National Laboratory, Leibniz Institut für Astrophysik Potsdam (AIP), Max-Planck-Institut für Astronomie (MPIA Heidelberg), Max-Planck-Institut für Astrophysik (MPA Garching), Max-Planck-Institut für Extraterrestrische Physik (MPE), National Astronomical Observatories of China, New Mexico State University, New York University, University of Notre Dame, Observatório Nacional / MCTI, The Ohio State University, Pennsylvania State University, Shanghai Astronomical Observatory, United Kingdom Participation Group, Universidad Nacional Autónoma de México, University of Arizona, University of Colorado Boulder, University of Oxford, University of Portsmouth, University of Utah, University of Virginia, University of Washington, University of Wisconsin, Vanderbilt University, and Yale University.
</font>
</div>

<font size=4>
SDSS criterion:

>The photometric pipeline version used for DR2 and later data (5_4) classifies objects as extended (“galaxy”) or point-like (“star”) based on the difference between the cmodel and PSF magnitude. An object is classified as extended if$$psfMag – cmodelMag > 0.145$$
>
>If satisfied, type is set to GALAXY for that band; otherwise, type is set to STAR. The global type objc_type is set according to the same criterion, applied to the summed fluxes from all bands in which the object is detected.
</font>

In [1]:
%matplotlib inline
import numpy
from matplotlib import pyplot
import astropy.io.ascii as ascii
from astroquery.sdss import SDSS

def downloadData(filename):
    object_query = "select top 10000 specObjID, psfMag_g, psfMag_r, psfMag_i, cModelMag_g, cModelMag_r, cModelMag_i,  class from SpecPhotoAll where psfMag_g > 10 order by newid()"
    result = SDSS.query_sql(object_query, cache=False)

    galaxyIndex = numpy.where(numpy.array(result['class'], dtype=str) == "GALAXY")[0]
    starIndex = numpy.where(numpy.array(result['class'], dtype=str) == "STAR")[0]
    qsoIndex = numpy.where(numpy.array(result['class'], dtype=str) == "QSO")[0]

    result['class'][galaxyIndex] = 1
    result['class'][starIndex] = -1
    result['class'][qsoIndex] = -1
    result = result.group_by('class')
    
    ascii.write(result, filename, format='csv')  

if False: downloadData("sdss_point_vs_extended.csv") #download data

result = ascii.read('sdss_point_vs_extended.csv', format='csv')

extended = numpy.where(numpy.array(result['class'].astype(int)) == 1)[0]
point = numpy.where(numpy.array(result['class'].astype(int)) == -1)[0]

colors = result['psfMag_r'] - result['cModelMag_r']

pyplot.figure(figsize=(16,5))
pyplot.hist(colors[extended], bins = 50, facecolor="r", alpha=0.5, range=(-0.5,4))
pyplot.hist(colors[point], bins = 50, facecolor="b", alpha=0.5, range=(-0.5,4))
pyplot.axvline(x=0.145, color="k")
pyplot.xlim(-0.5,4)
pyplot.show()

/home/muha/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/muha/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'astroquery'

In [ ]:
def calculateError(threshold, colors, classes):
    return (numpy.where((colors > threshold) & (classes == 1))[0].shape[0] +\
           numpy.where((colors <= threshold) & (classes == -1))[0].shape[0]) / colors.shape[0]
    
print ("%2.2f%% correctly classified" % (calculateError(0.145, colors, numpy.array(result['class'].astype(int)))*100))

In [ ]:
from scipy.optimize import minimize

def optimalThreshold(threshold):
    return 1-calculateError(threshold, colors, numpy.array(result['class'].astype(int)))

bestThreshold = minimize(optimalThreshold, 0, method='nelder-mead').x

print ("%2.2f%% correctly classified with a threshold value of %f" % ((calculateError(bestThreshold, colors, numpy.array(result['class'].astype(int)))*100) , bestThreshold))

pyplot.figure(figsize=(16,5))
pyplot.hist(colors[extended], bins = 50, facecolor="r", alpha=0.5, range=(-0.5,4))
pyplot.hist(colors[point], bins = 50, facecolor="b", alpha=0.5, range=(-0.5,4))
pyplot.axvline(x=0.145, color="k")
pyplot.axvline(x=bestThreshold, color="g")
pyplot.xlim(-0.5,4)
pyplot.show()

<br/><br/><br/><br/><font color="113388" size="6">4. Classification with reference data</font><br/><br/>

<font size=4>
We want to learn to classify by learning from data. Simple but powerful approach is to use a k-nearest-neighbor model.<br/>
    
$$\hat Y(x) = \frac{1}{k} \sum_{x_i \in N_k(x)} y_i$$

with $\hat Y$ being the prediction for an input $x$, calculated as the dominant class in the class values $y_i$ of the $k$ most similar objects retrieved by the neighborhood function $N_k(x)$.
</font>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about k-nearewst neighbors

* https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm
* https://web.stanford.edu/~hastie/ElemStatLearn/
* https://www.youtube.com/watch?v=UqYde-LULfs
* https://www.coursera.org/lecture/big-data-machine-learning/k-nearest-neighbors-MamQ9

read more about spatial neighborhood function implementation via kd-tree

* https://en.wikipedia.org/wiki/K-d_tree
* https://www.geeksforgeeks.org/k-dimensional-tree/
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html

</div>


In [ ]:
import numpy.random

numpy.random.seed(1234)

n = 25
marker = ["x","o"]

x1 = numpy.random.uniform(0,1,n)
y1 = numpy.random.uniform(0,0.5,n)

y2 = numpy.random.uniform(0.5,1,n)
x2 = numpy.random.uniform(0,1,n)

X = numpy.concatenate((x1,x2,y1,y2)).reshape(2,-1).T
Y = numpy.concatenate((numpy.ones(n)*-1, numpy.ones(n)))

pyplot.figure(figsize=(16,10))
#pyplot.scatter(X[:,0],X[:,1],c=Y,s=100)
pyplot.scatter(x1,y1,c="r",marker="x",s=100)
pyplot.scatter(x2,y2,c="b",marker="o",s=100)

from scipy.spatial import KDTree

if False:
    a = 0.7
    b = 0.65
    pyplot.scatter(a,b,c="g",marker="*",s=400)
    
    k = 5
    myTree = KDTree(X, leafsize=3)
    neighbors = myTree.query([a,b], k=k)
    print (neighbors, Y[neighbors[1]])
    #pyplot.scatter(X[:,0][neighbors[1]], X[:,1][neighbors[1]], marker="o", s=400, alpha=0.5)
    
pyplot.show()

In [ ]:
myTree = KDTree(X, leafsize=3)

def getClass(x, k):
    neighbors = myTree.query(x, k=k)
    if numpy.mean(Y[neighbors[1]]) > 0.0:
        return 1
    return -1

stepSize = 0.5
coordinates = numpy.mgrid[0:1+stepSize:stepSize, 0:1+stepSize:stepSize].reshape(2,-1).T

k = 5

classes = [getClass(c, k) for c in coordinates]

map = numpy.flipud(numpy.asarray(classes).reshape(int(1.0/stepSize) + 1,int(1.0/stepSize) + 1).T)

pyplot.figure(figsize=(16,10))
pyplot.scatter(x1,y1,c="r",marker="x",s=100)
pyplot.scatter(x2,y2,c="b",marker="o",s=100)
pyplot.imshow(map, extent=[0-stepSize/2.0,1+stepSize/2.0,0-stepSize/2.0,1+stepSize/2.0], interpolation="nearest", alpha=0.2, aspect="auto")

pyplot.axhline(0.5,c="k") # overfitting !!!!

pyplot.show()

##### <br/><br/>
<font size=4>
Let's go back to the real data example and use more dimensions than we could visualize.

So many features to pick from:
<br/><br/>
    
    
* $mag_{psf}^g$, $mag_{psf}^r$, $mag_{psf}^i$
* $mag_{model}^g$, $mag_{model}^r$, $mag_{model}^i$
* $mag_{psf}^g - mag_{psf}^r$, $mag_{psf}^r - mag_{psf}^i$
* $mag_{model}^g - mag_{model}^r$, $mag_{model}^r - mag_{model}^i$
* $mag_{psf}^g - mag_{model}^g$, $mag_{psf}^r - mag_{model}^r$, $mag_{psf}^i - mag_{model}^i$
</font>

<br/><br/>

In [ ]:
data = ascii.read('sdss_point_vs_extended.csv', format='csv')

X = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']))).reshape(3,-1).T
Y = numpy.array(data['class'])

myTree = KDTree(X, leafsize=10)
k = 1

predictedClasses = numpy.array([getClass(x, k) for x in X])

print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Y)/2)/Y.shape[0]) * 100, "% correctly classified")

<br/><br/><br/><br/><font color="113388" size="6">5. Evaluation</font><br/><br/>

<font size=4>
We need a better way to evaluate the performance of our algorithm. We want to compare predicted with the real classes.</font><br/>

<table><tr><td>
    <font size=7>True Positive</font>
    <font size=4><br/>objects from the +1 class correctly classified</font></td><td>
    <font size=7>False Positive</font>
    <font size=4><br/>objects from the +1 class NOT correctly classified</font></td></tr>
  <tr><td>
    <font size=7>False Negative</font>
    <font size=4><br/>objects from the -1 class NOT correctly classified</font></td><td>
    <font size=7>True Negative</font>
    <font size=4><br/>objects from the -1 class correctly classified</font></td></tr></table>
</font>

<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about simple class evaluation

* https://en.wikipedia.org/wiki/False_positives_and_false_negatives
* https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative
* https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
def evaluateResults(predictions, correctValues):
    truePositive = numpy.where(predictions[numpy.where(correctValues == 1)[0]]==1)[0].shape
    falsePositive = numpy.where(predictions[numpy.where(correctValues == 1)[0]]==-1)[0].shape
    trueNegative = numpy.where(predictions[numpy.where(correctValues == -1)[0]]==-1)[0].shape
    falseNegative = numpy.where(predictions[numpy.where(correctValues == -1)[0]]==1)[0].shape
    return truePositive, falsePositive, falseNegative, trueNegative

confusionMatrix = numpy.array(evaluateResults(predictedClasses, Y))
print ((confusionMatrix/Y.shape[0]*100.0).reshape(2,2))
#optimize k

<br/><br/><font size=4>
Now let's simulate we observe data the next night and see how well we perform!</font><br/><br/>

In [ ]:
if False: downloadData("sdss_point_vs_extended_2.csv") #download data
    
data = ascii.read('sdss_point_vs_extended_2.csv', format='csv')

Xnew = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']))).reshape(3,-1).T
Ynew = numpy.array(data['class'])

predictedClasses = numpy.array([getClass(x, k) for x in Xnew])
confusionMatrix = numpy.array(evaluateResults(predictedClasses, Ynew))

print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Ynew)/2)/Ynew.shape[0]) * 100, "% correctly classified")
print ((confusionMatrix/Ynew.shape[0]*100.0).reshape(2,2))

<br/><br/><br/><br/><font color="113388" size="6">6. Validation</font><br/><br/>

<font size=4>
What went wrong? We used the same data for training and for evaluation.

What is a good model? How can we properly validate, whether the model generalized correctly or not.

Do exactly what we did in the experiment:

* select data for training
* select data for evaluating
* make sure that those sets are independent ... think of observing the next night

</font>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about validation

* https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets
* https://en.wikipedia.org/wiki/Cross-validation_(statistics)
* https://www.geeksforgeeks.org/cross-validation-machine-learning/
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

</div>

<font size=4>
Let's go back to synthetic data!
</font>

In [ ]:
numpy.random.seed(1234)

n = 8

x1 = numpy.concatenate((numpy.random.uniform(0,0.6,n//2),numpy.random.uniform(0.6,1,n//2)))
y1 = numpy.concatenate((numpy.random.uniform(0,0.5,n//2),numpy.random.uniform(0.0,1,n//2))) 
y2 = numpy.random.uniform(0.5,1,n)
x2 = numpy.random.uniform(0,0.6,n)

x3 = numpy.concatenate((numpy.random.uniform(0,0.6,n//2),numpy.random.uniform(0.6,1,n//2)))
y3 = numpy.concatenate((numpy.random.uniform(0,0.5,n//2),numpy.random.uniform(0.0,1,n//2))) 
y4 = numpy.random.uniform(0.5,1,n)
x4 = numpy.random.uniform(0,0.6,n)

Xtrain = numpy.concatenate((x1,x2,y1,y2)).reshape(2,-1).T
Ytrain = numpy.concatenate((numpy.ones(n)*-1, numpy.ones(n)))

Xvalidation = numpy.concatenate((x3,x4,y3,y4)).reshape(2,-1).T
Yvalidation = numpy.concatenate((numpy.ones(n)*-1, numpy.ones(n)))

k = 5
myTree = KDTree(Xtrain, leafsize=3)

def getClass(x, k):
    neighbors = myTree.query(x, k=k)
    if numpy.mean(Ytrain[neighbors[1]]) > 0.0:
        return 1
    return -1

stepSize = 0.02
coordinates = numpy.mgrid[0:1+stepSize:stepSize, 0:1+stepSize:stepSize].reshape(2,-1).T

classes = [getClass(c, k) for c in coordinates]
map = numpy.flipud(numpy.asarray(classes).reshape(int(1.0/stepSize) + 1,int(1.0/stepSize) + 1).T)

predictedClasses = numpy.array([getClass(x, k) for x in Xvalidation])
confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidation))

pyplot.figure(figsize=(16,10))
pyplot.imshow(map, extent=[0-stepSize/2.0,1+stepSize/2.0,0-stepSize/2.0,1+stepSize/2.0], interpolation="nearest", alpha=0.2, aspect="auto")
pyplot.scatter(x1,y1,c="r",marker="x",s=100)
pyplot.scatter(x2,y2,c="b",marker="o",s=100)
pyplot.scatter(x3,y3,c="r",marker="+",s=100, alpha=0.5)
pyplot.scatter(x4,y4,c="b",marker="p",s=100, alpha=0.5)
#pyplot.scatter(Xvalidation[:,0],Xvalidation[:,1],c=predictedClasses, marker="o", s=400, alpha=0.25)

pyplot.show()

print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidation)/2)/Yvalidation.shape[0]) * 100, "% correctly classified")
print ((confusionMatrix/Yvalidation.shape[0]*100.0).reshape(2,2))
#optimize k

In [ ]:
x5 = numpy.concatenate((numpy.random.uniform(0,0.6,n//2),numpy.random.uniform(0.6,1,n//2)))
y5 = numpy.concatenate((numpy.random.uniform(0,0.5,n//2),numpy.random.uniform(0.0,1,n//2))) 
y6 = numpy.random.uniform(0.5,1,n)
x6 = numpy.random.uniform(0,0.6,n)

Xtest = numpy.concatenate((x5,x6,y5,y6)).reshape(2,-1).T
Ytest = numpy.concatenate((numpy.ones(n)*-1, numpy.ones(n)))

predictedClasses = numpy.array([getClass(x, k) for x in Xtest])
confusionMatrix = numpy.array(evaluateResults(predictedClasses, Ytest))

print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Ytest)/2)/Ytest.shape[0]) * 100, "% correctly classified")
print ((confusionMatrix/Ytest.shape[0]*100.0).reshape(2,2))

<font size=4>Let's apply it to the real data example and check the results!</font>

In [ ]:
data = ascii.read('sdss_point_vs_extended.csv', format='csv')

Xnew = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']))).reshape(3,-1).T
Ynew = numpy.array(data['class'])

n = Ynew.shape[0]
Xtrain = Xnew[:n//3]
Ytrain = Ynew[:n//3]
Xvalidate = Xnew[n//3:n//3*2]
Yvalidate = Ynew[n//3:n//3*2]
Xtest = Xnew[-n//3:]
Ytest = Ynew[-n//3:]

k = 5
myTree = KDTree(Xtrain, leafsize=5)

def getClass(x, k):
    neighbors = myTree.query(x, k=k)
    if numpy.mean(Ytrain[neighbors[1]]) > 0.0:
        return 1
    return -1

predictedClasses = numpy.array([getClass(x, k) for x in Xvalidate])
confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidate))
print ("validation:")
print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidate)/2)/Yvalidate.shape[0]) * 100, "% correctly classified")
print ((confusionMatrix/Yvalidate.shape[0]*100.0).reshape(2,2))

predictedClasses = numpy.array([getClass(x, k) for x in Xtest])
confusionMatrix = numpy.array(evaluateResults(predictedClasses, Ytest))
print ("testing:")
print ((1.0 - numpy.sum(numpy.abs(predictedClasses - Ytest)/2)/Ytest.shape[0]) * 100, "% correctly classified")
print ((confusionMatrix/Ytest.shape[0]*100.0).reshape(2,2))

<font size=4>
What went wrong? We should use a better validation technique to figure out what happened. There are different techniques available. One of the most common ones in machine learning is:
</font>

<br/><br/>

<center><h1>k-fold cross validation</h1></center><br/><br/>

<font size=4>
5-fold cross validation as an example:<br/>
    
<table style="font-size:20px;">
    <tr><td>run #1</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#fcc">validate on this</td></tr>
    <tr><td>run #2</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#fcc">validate on this</td>
        <td bgcolor="#ccf">train on this</td></tr>
    <tr><td>run #3</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#fcc">validate on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td></tr>
    <tr><td>run #4</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#fcc">validate on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td></tr>
    <tr><td>run #5</td>
        <td bgcolor="#fcc">validate on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td>
        <td bgcolor="#ccf">train on this</td></tr>
</table>

</font>

In [ ]:
from sklearn.model_selection import KFold

data = ascii.read('sdss_point_vs_extended.csv', format='csv')

X = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']))).reshape(3,-1).T
Y = numpy.array(data['class'])

def getClass(tree, labels, x, k):
    neighbors = tree.query(x, k=k)
    if numpy.mean(labels[neighbors[1]]) > 0.0:
        return 1
    return -1

kFold = KFold(n_splits=5, shuffle=False)
k = 5
i = 1

results = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]
   
    myTree = KDTree(Xtrain, leafsize=5)
    predictedClasses = numpy.array([getClass(myTree, Ytrain, x, k) for x in Xvalidate])
    confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidate))
    results.append((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidate)/2)/Yvalidate.shape[0]))
    print ("validation run %2d: %2.2f%% correctly classified" %(i, results[-1] * 100))
    print ((confusionMatrix/Yvalidate.shape[0]*100.0).reshape(2,2))
    i = i + 1
print ("####################################")
print ("mean performance: %2.2f%% standard deviation: %2.2f%%" % (numpy.mean(results)*100.0, numpy.std(results)*100.0))

<br/><br/>
<font size=4>
We used ordered data with the point sources first. Therefore we did not have representative training data and we just evaluated the performance on extended sources. By enabling the shuffling, we make sure that the individual folds are randomly populated!
    
Next, let's use more features!
</font>
<br/><br/>

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize

data = ascii.read('sdss_point_vs_extended.csv', format='csv')

X = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']),
                       numpy.array(data['psfMag_g'] - data['psfMag_r']),
                       numpy.array(data['psfMag_r'] - data['psfMag_i']),
                       numpy.array(data['cModelMag_g'] - data['cModelMag_r']),
                       numpy.array(data['cModelMag_r'] - data['cModelMag_i']),
                       numpy.array(data['psfMag_g']),
                       numpy.array(data['psfMag_r']),
                       numpy.array(data['psfMag_i']),
                       numpy.array(data['cModelMag_g']),
                       numpy.array(data['cModelMag_r']),
                       numpy.array(data['cModelMag_i']))).reshape(13,-1).T
Y = numpy.array(data['class'])

def getClass(tree, labels, x, k):
    neighbors = tree.query(x, k=k)
    if numpy.mean(labels[neighbors[1]]) > 0.0:
        return 1
    return -1

kFold = KFold(n_splits=5, shuffle=True)
k = 5
i = 1

results = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]
   
    myTree = KDTree(Xtrain, leafsize=5)
    predictedClasses = numpy.array([getClass(myTree, Ytrain, x, k) for x in Xvalidate])
    confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidate))
    results.append((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidate)/2)/Yvalidate.shape[0]))
    print ("validation run %2d: %2.2f%% correctly classified" %(i, results[-1] * 100))
    print ((confusionMatrix/Yvalidate.shape[0]*100.0).reshape(2,2))
    i = i + 1
print ("####################################")
print ("mean performance: %2.2f%% standard deviation: %2.2f%%" % (numpy.mean(results)*100.0, numpy.std(results)*100.0))

In [ ]:
if True: #normalize the data
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - numpy.min(X[:,i])) / (numpy.max(X[:,i]) - numpy.min(X[:,i]))
    
kFold = KFold(n_splits=5, shuffle=True)
k = 5
i = 1

results = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]
   
    myTree = KDTree(Xtrain, leafsize=5)
    predictedClasses = numpy.array([getClass(myTree, Ytrain, x, k) for x in Xvalidate])
    confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidate))
    results.append((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidate)/2)/Yvalidate.shape[0]))
    print ("validation run %2d: %2.2f%% correctly classified" %(i, results[-1] * 100))
    print ((confusionMatrix/Yvalidate.shape[0]*100.0).reshape(2,2))
    i = i + 1
print ("####################################")
print ("mean performance: %2.2f%% standard deviation: %2.2f%%" % (numpy.mean(results)*100.0, numpy.std(results)*100.0))

<br/><br/><br/><br/><font color="113388" size="6">7. Unbalanced training data</font><br/><br/>


<font size=4>
There are to many extended source in comparison to compact sources in the sample. Think of 99% of class A and 1% of class B. Always predicting A should give 99% correct classifications, even though the model did not generalize.
    
Let's use a balanced data set and calculate some correlation coefficients!
</font>

<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about correlation coefficients and unbalanced training sets

* https://en.wikipedia.org/wiki/Matthews_correlation_coefficient
* https://lettier.github.io/posts/2016-08-05-matthews-correlation-coefficient.html
* https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
* https://towardsdatascience.com/deep-learning-unbalanced-training-data-solve-it-like-this-6c528e9efea6
* https://www.kdnuggets.com/2017/06/7-techniques-handle-imbalanced-data.html

</div>

<br/><br/>

In [ ]:
import math

def calculateMCC(confusionMatrix):
    truePositive = confusionMatrix[0]
    falsePositive = confusionMatrix[1]
    falseNegative = confusionMatrix[2]
    trueNegative = confusionMatrix[3]
    return (truePositive*trueNegative - falsePositive*falseNegative) / \
           math.sqrt((truePositive+falsePositive)*(truePositive+falseNegative)*(trueNegative+falsePositive)*(trueNegative+falseNegative))

print (calculateMCC(confusionMatrix))

In [ ]:
from astropy.table import vstack

def downloadBalancedData(filename):
    galaxy_query = "select top 5000 specObjID, psfMag_g, psfMag_r, psfMag_i, cModelMag_g, cModelMag_r, cModelMag_i,  class from SpecPhotoAll where psfMag_g > 10 and class='GALAXY' order by newid()"
    resultG = SDSS.query_sql(galaxy_query, cache=False)
    quasar_query = "select top 5000 specObjID, psfMag_g, psfMag_r, psfMag_i, cModelMag_g, cModelMag_r, cModelMag_i,  class from SpecPhotoAll where psfMag_g > 10 and (class='STAR' or class='QSO') order by newid()"
    resultQ = SDSS.query_sql(quasar_query, cache=False)
    
    result = vstack((resultG,resultQ))

    galaxyIndex = numpy.where(numpy.array(result['class'], dtype=str) == "GALAXY")[0]
    starIndex = numpy.where(numpy.array(result['class'], dtype=str) == "STAR")[0]
    qsoIndex = numpy.where(numpy.array(result['class'], dtype=str) == "QSO")[0]

    result['class'][galaxyIndex] = 1
    result['class'][starIndex] = -1
    result['class'][qsoIndex] = -1
    result = result.group_by('class')
    
    ascii.write(result, filename, format='csv')
    
if False: downloadBalancedData("sdss_point_vs_extended_balanced.csv") #download data

data = ascii.read('sdss_point_vs_extended_balanced.csv', format='csv')

X = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']),
                       numpy.array(data['psfMag_g'] - data['psfMag_r']),
                       numpy.array(data['psfMag_r'] - data['psfMag_i']),
                       numpy.array(data['cModelMag_g'] - data['cModelMag_r']),
                       numpy.array(data['cModelMag_r'] - data['cModelMag_i']),
                       numpy.array(data['psfMag_g']),
                       numpy.array(data['psfMag_r']),
                       numpy.array(data['psfMag_i']),
                       numpy.array(data['cModelMag_g']),
                       numpy.array(data['cModelMag_r']),
                       numpy.array(data['cModelMag_i']))).reshape(13,-1).T
Y = numpy.array(data['class'])

if True:
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - numpy.min(X[:,i])) / (numpy.max(X[:,i]) - numpy.min(X[:,i]))
    
kFold = KFold(n_splits=5, shuffle=True)
k = 5
i = 1

results = []
mccs = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]
   
    myTree = KDTree(Xtrain, leafsize=10)
    predictedClasses = numpy.array([getClass(myTree, Ytrain, x, k) for x in Xvalidate])
    confusionMatrix = numpy.array(evaluateResults(predictedClasses, Yvalidate))
    results.append((1.0 - numpy.sum(numpy.abs(predictedClasses - Yvalidate)/2)/Yvalidate.shape[0]))
    mccs.append(calculateMCC(confusionMatrix))
    print ("validation run %2d: %2.2f%% correctly classified, MCC = %2.2f" %(i, results[-1] * 100,mccs[-1]))
    print ((confusionMatrix/Yvalidate.shape[0]*100.0).reshape(2,2))
    i = i + 1
print ("####################################")
print ("mean performance: %2.2f%% standard deviation: %2.2f%%" % (numpy.mean(results)*100.0, numpy.std(results)*100.0))
print ("mean MCC: %2.2f standard deviation: %2.2f" % (numpy.mean(mccs), numpy.std(mccs)))

<br/><br/><br/><br/><font color="113388" size="6">8. Regression with reference data</font><br/><br/>


<font size=4>
After we learned how to classify objects it's time to start estimating their redshift. Use the same algorithm but don't use a majority vote.

We want to learn to estimate by learning from data. Simple but powerful approach is to use a k-nearest-neighbor model.

$$\hat Y(x) = \frac{1}{k} \sum_{x_i \in N_k(x)} y_i$$

with $\hat Y$ being the prediction for an input $x$, calculated as the mean of the values $y_i$ of the $k$ most similar objects retrieved by the neighborhood function $N_k(x)$.

</font>

<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about nearest neighbor regression

* https://web.stanford.edu/~hastie/ElemStatLearn/
* https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
* https://www.coursera.org/lecture/ml-regression/k-nearest-neighbors-regression-8pPF9

</div>

<br/><br/>

In [ ]:
def downloadRedshiftData(filename):
    object_query = "select top 10000 specObjID, psfMag_u, psfMag_g, psfMag_r, psfMag_i, psfMag_z, cModelMag_u, cModelMag_g, cModelMag_r, cModelMag_i, cModelMag_z, z from SpecPhotoAll where psfMag_g > 10 and class='QSO' and z<5 order by newid()"
    result = SDSS.query_sql(object_query, cache=False)
    ascii.write(result, filename, format='csv')
    
if False: downloadRedshiftData("sdss_redshift.csv") #download data

data = ascii.read('sdss_redshift.csv', format='csv')
X = numpy.concatenate((numpy.array(data['psfMag_g'] - data['cModelMag_g']),
                       numpy.array(data['psfMag_r'] - data['cModelMag_r']),
                       numpy.array(data['psfMag_i'] - data['cModelMag_i']),
                       numpy.array(data['psfMag_u'] - data['psfMag_g']),
                       numpy.array(data['psfMag_g'] - data['psfMag_r']),
                       numpy.array(data['psfMag_r'] - data['psfMag_i']),
                       numpy.array(data['psfMag_i'] - data['psfMag_z']),
                       numpy.array(data['cModelMag_u'] - data['cModelMag_g']),
                       numpy.array(data['cModelMag_g'] - data['cModelMag_r']),
                       numpy.array(data['cModelMag_r'] - data['cModelMag_i']),
                       numpy.array(data['cModelMag_i'] - data['cModelMag_z']),
                       numpy.array(data['psfMag_r']),
                       numpy.array(data['cModelMag_r']))).reshape(13,-1).T
Y = numpy.array(data['z'])
names = ["psf_g-model_g","psf_r-model_r","psf_i-model_i","psf_u-psf_g","psf_g-psf_r","psf_r-psf_i","psf_i-psf_z","model_u-model_g","model_g-model_r","model_r-model_i","model_i-model_z","psf_r","model_r"]

if True:
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - numpy.min(X[:,i])) / (numpy.max(X[:,i]) - numpy.min(X[:,i]))
        
def getPrediction(tree, labels, x, k):
    neighbors = tree.query(x, k=k)
    return numpy.mean(labels[neighbors[1]])
    
kFold = KFold(n_splits=5, shuffle=True)
k = 5
i = 1

results = []
specZ = []
photoZ = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]
   
    myTree = KDTree(Xtrain, leafsize=5)
    predictedRedshifts = numpy.array([getPrediction(myTree, Ytrain, x, k) for x in Xvalidate])
    results.append(numpy.sqrt(numpy.mean(numpy.square(predictedRedshifts - Yvalidate))))
    specZ.append(Yvalidate)
    photoZ.append(predictedRedshifts)
    print ("validation run %2d: RMSE: %2.2f" %(i, results[-1]))
    i = i + 1
print ("####################################")
print ("mean RMSE: %2.2f standard deviation: %2.2f" % (numpy.mean(results), numpy.std(results)))

specZ = numpy.array(specZ).flatten()
photoZ = numpy.array(photoZ).flatten()
pyplot.figure(figsize=(10,10))
pyplot.scatter(specZ, photoZ, alpha=0.1)
pyplot.xlim(0,numpy.max(specZ))
pyplot.ylim(0,numpy.max(specZ))
pyplot.show()

<br/><br/><br/><br/><font color="113388" size="6">9. How to further optimize - other algorithms / decision trees</font><br/><br/>

<font size=4>
After we learned about validation and scoring, there are many thinks to optimize:<br/>

* features
* distance function
* weights
* preprocessing
* hyperparameters

How about a different algorithm: Let's use decision trees.
</font>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about decision trees


* https://en.wikipedia.org/wiki/Decision_tree_learning
* https://scikit-learn.org/stable/modules/tree.html
* https://medium.com/deep-math-machine-learning-ai/chapter-4-decision-trees-algorithms-b93975f7a1f1
* https://web.stanford.edu/~hastie/Papers/ESLII.pdf page 305[324]

</div>
<br/><br/>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

kFold = KFold(n_splits=5, shuffle=True)

i = 1

results = []
specZ = []
photoZ = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]

    myTree = DecisionTreeRegressor(max_depth=20)
    myTree.fit(Xtrain, Ytrain)
    predictedRedshifts = myTree.predict(Xvalidate)
    results.append(numpy.sqrt(numpy.mean(numpy.square(predictedRedshifts - Yvalidate))))
    specZ.append(Yvalidate)
    photoZ.append(predictedRedshifts)
    print ("validation run %2d: RMSE: %2.2f" %(i, results[-1]))
    i = i + 1
print ("####################################")
print ("mean RMSE: %2.2f standard deviation: %2.2f" % (numpy.mean(results), numpy.std(results)))

specZ = numpy.array(specZ).flatten()
photoZ = numpy.array(photoZ).flatten()

pyplot.figure(figsize=(10,10))
pyplot.scatter(specZ, photoZ, alpha=0.1)
pyplot.xlim(0,numpy.max(specZ))
pyplot.ylim(0,numpy.max(specZ))
pyplot.show()

In [ ]:
print (numpy.sort(myTree.feature_importances_))
print (numpy.array(names)[numpy.argsort(myTree.feature_importances_)])

<br/><br/><br/><br/><font color="113388" size="6">10. Ensemble learning - random forest</font><br/><br/>

<font size=4>
Let's use the results of multiple experts to reduce variance, by combine predictors with uncorrelated errors. In this case we will use an ensemble of decision trees also known as random forest.
</font>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">
read more about ensemble<br/>

* https://de.wikipedia.org/wiki/Ensemble_learning
* https://en.wikipedia.org/wiki/Random_forest
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
* https://web.stanford.edu/~hastie/Papers/ESLII.pdf page 587[606], 605[624]
* http://www.jmlr.org/papers/volume5/chawla04a/chawla04a.pdf
</div>


In [ ]:
from sklearn.ensemble import RandomForestRegressor

kFold = KFold(n_splits=5, shuffle=True)
i = 1

results = []
specZ = []
photoZ = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]

    myForest = RandomForestRegressor(n_estimators=164)
    myForest.fit(Xtrain, Ytrain)

    predictedRedshifts = myForest.predict(Xvalidate)
    results.append(numpy.sqrt(numpy.mean(numpy.square(predictedRedshifts - Yvalidate))))
    specZ.append(Yvalidate)
    photoZ.append(predictedRedshifts)
    print ("validation run %2d: RMSE: %2.2f" %(i, results[-1]))
    i = i + 1
print ("####################################")
print ("mean RMSE: %2.2f standard deviation: %2.2f" % (numpy.mean(results), numpy.std(results)))

specZ = numpy.array(specZ).flatten()
photoZ = numpy.array(photoZ).flatten()
pyplot.figure(figsize=(10,10))
pyplot.scatter(specZ, photoZ, alpha=0.1)
pyplot.xlim(0,numpy.max(specZ))
pyplot.ylim(0,numpy.max(specZ))
pyplot.show()

<br/><br/><br/><br/><font color="113388" size="6">11. Doing science with all the things presented above</font><br/><br/>

<font size=4>
Using a GPU-based, kNN ensemble, multi-massive feature selection approach we found the following features to be very powerful. Just use the best 7 out of 10 for the next experiment: 

$$mag_{petro}^i/mag_{psf}^i$$
$$\hat {mag}_{psf}^g - \hat {mag}_{model}^u$$
$$\hat {mag}_{exp}^i / \hat {mag}_{psf}^r$$
$$\sqrt { {\sigma_{model}^r}^2 + {\sigma_{dev}^r}^2}$$
$$\hat {mag}_{psf}^r / \hat {mag}_{exp}^g$$
$$\hat {mag}_{psf}^i / \hat {mag}_{model}^z$$
$${mag}_{psf}^i - {mag}_{dev}^i$$
</font>

<br/><br/>
<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">
    
read more about feature selection

* https://arxiv.org/abs/1803.10032
* https://arxiv.org/abs/1310.1976

</div>

<br/><br/>

![Feature Selection](FeatureSelection.png "Feature Selection")

In [ ]:
def downloadRedshiftDataFeatures(filename):
    object_query = "select top 10000 specObjID, (petroMag_i/psfMag_i) as fa, ((psfMag_g-extinction_g)-(modelMag_u-extinction_u)) as fb, ((cModelMag_i-extinction_i)/(psfMag_r-extinction_r)) as fc, sqrt(square(modelMagErr_r) + square(cModelMagErr_r)) as fd, ((psfMag_r-extinction_r)/(cModelMag_g-extinction_g)) as fe, ((psfMag_i-extinction_i)/(modelMag_z-extinction_z)) as ff, (psfMag_i-cModelMag_i) as fg, z from SpecPhotoAll where psfMag_g > 10 and class='QSO' and z<5 order by newid()"
    result = SDSS.query_sql(object_query, cache=False)
    ascii.write(result, filename, format='csv')
    
if False: downloadRedshiftDataFeatures("sdss_redshift_best_features.csv") #download data

data = ascii.read('sdss_redshift_best_features.csv', format='csv')
X = numpy.concatenate((numpy.array(data['fa']),numpy.array(data['fb']),numpy.array(data['fc']),numpy.array(data['fd']),numpy.array(data['fe']),numpy.array(data['ff']),numpy.array(data['fg']))).reshape(7,-1).T
Y = numpy.array(data['z'])

kFold = KFold(n_splits=5, shuffle=True)
i = 1

results = []
specZ = []
photoZ = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]

    myForest = RandomForestRegressor(n_estimators=64)
    myForest.fit(Xtrain, Ytrain)

    predictedRedshifts = myForest.predict(Xvalidate)
    results.append(numpy.sqrt(numpy.mean(numpy.square(predictedRedshifts - Yvalidate))))
    specZ.append(Yvalidate)
    photoZ.append(predictedRedshifts)
    print ("validation run %2d: RMSE: %2.2f" %(i, results[-1]))
    i = i + 1
print ("####################################")
print ("mean RMSE: %2.2f standard deviation: %2.2f" % (numpy.mean(results), numpy.std(results)))

specZ = numpy.array(specZ).flatten()
photoZ = numpy.array(photoZ).flatten()
pyplot.figure(figsize=(10,10))
pyplot.scatter(specZ, photoZ, alpha=0.1)
pyplot.xlim(0,numpy.max(specZ))
pyplot.ylim(0,numpy.max(specZ))
pyplot.show()


<br/><br/><br/><br/><font color="113388" size="6">12. Artificial Neural Networks</font><br/><br/>

<font size=4>
How about a different algorithm: Let's use artificial neural networks. In this example Multi Layer Perceptron (MLP).<br/>
    
Completely different to the methods before:

* generalizing
* concept of training<br/>
  by updating weights and biases of the network - backpropagation
* concept of loss-function
* very quick to calculate once trained - very time consuming training


many more hyper-parameter / options

* number of layers
* size of layers
* activation functions


* different training strategies / regularization

</font>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about artificial neural networks


* https://en.wikipedia.org/wiki/Artificial_neural_network
* https://www.youtube.com/watch?v=aircAruvnKk
* https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6
* https://iamtrask.github.io/2015/07/12/basic-python-network/
* https://www.kdnuggets.com/2016/10/beginners-guide-neural-networks-python-scikit-learn.html
* https://scikit-learn.org/stable/modules/neural_networks_supervised.html
* https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

</div>
<br/><br/>

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + numpy.exp(-x))

def sigmoidDiff(x):
    return x * (1.0 - x)

xplot = numpy.arange(-10,10,0.1)
pyplot.figure(figsize=(16,5))
ax1 = pyplot.gca()
ax1.scatter(xplot, sigmoid(xplot), c="r") # numpy.tanh
ax2 = ax1.twinx()
ax2.scatter(xplot, sigmoidDiff(sigmoid(xplot)))
pyplot.show()

In [ ]:
from sklearn.neural_network import MLPRegressor

myNet = MLPRegressor(hidden_layer_sizes = (15,7),
                     activation='relu',#logistic, tanh, relu
                     solver='sgd', #sgd, lbfgs
                     learning_rate_init = 0.001,
                     tol=1e-4,
                     verbose=False)

kFold = KFold(n_splits=5, shuffle=True)
i = 1

results = []
specZ = []
photoZ = []

for trainIndex, validateIndex in kFold.split(X):
    Xtrain, Xvalidate = X[trainIndex], X[validateIndex]
    Ytrain, Yvalidate = Y[trainIndex], Y[validateIndex]

    myNet.fit(Xtrain, Ytrain)

    predictedRedshifts = myNet.predict(Xvalidate)
    results.append(numpy.sqrt(numpy.mean(numpy.square(predictedRedshifts - Yvalidate))))
    specZ.append(Yvalidate)
    photoZ.append(predictedRedshifts)
    print ("validation run %2d: RMSE: %2.2f" %(i, results[-1]))
    i = i + 1
print ("####################################")
print ("mean RMSE: %2.2f standard deviation: %2.2f" % (numpy.mean(results), numpy.std(results)))

specZ = numpy.array(specZ).flatten()
photoZ = numpy.array(photoZ).flatten()
pyplot.figure(figsize=(10,10))
pyplot.scatter(specZ, photoZ, alpha=0.1)
pyplot.xlim(0,numpy.max(specZ))
pyplot.ylim(0,numpy.max(specZ))
pyplot.show()


<br/><br/><br/><br/><font color="113388" size="6">13. What's next</font><br/><br/>

<font size=4>you learned about:</font>
    
<center><h1>optimization is machine learning</h1></center>
<center><h1>classification with explicit criterion</h1></center>
<center><h1>classification with reference data</h1></center>
<center><h1>regression with reference data</h1></center>
<center><h1>evaluation</h1></center>
<center><h1>validation</h1></center>
<center><h1>unbalanced data</h1></center>
<center><h1>ensemble strategies</h1></center>
<center><h1>kNN, decision tree, random forest, artificial neural networks</h1></center><br/><br/>

<font size=4>you can learn more about:</font>

<center><h1>deep learning, recurrent neural networks, autoencoder</h1></center>
<center><h1>unsupervised methods</h1><br/>
clustering, dimensionality reduction, outlier detection, kernel methods</center>
<center><h1>Bayesian methods / inference</h1></center>
<center><h1>information theory</h1></center>
<center><h1>scoring functions</h1></center>
<center><h1>knowledge transfer / domain shift / active learning<br/>
transfer learning / reinforcement learning<br/>
    online learning / learning ... </h1></center>
<br/><br/>

<div style="background-color:#f0f0ff;padding: 10px;outline-color: #0000ff;outline-width: 1px;outline-style: solid;width:40%;position:relative;left:60%;">

read more about machine learning

* https://web.stanford.edu/~hastie/Papers/ESLII.pdf
* https://see.stanford.edu/course/cs229
* http://lmgtfy.com/?q=machine+learning

</div>

In [ ]:
words = numpy.array(["attention", "sleeping", "your", "not", "thanks", "boring", "is", "for"])
index = numpy.array([4,7,2,0])
print (words[index])